In [74]:
import pandas as  pd

In [75]:
df = pd.read_csv('data/lob/full_lob_head.csv', index_col=0)

In [76]:

df.sort_values(by=['Date','Timestamp'])

df = df.pivot_table(index=['Timestamp', 'Exchange', 'Date'], 
                              columns='Order Type', 
                              values=['Price', 'Quantity'])

df.columns = [' '.join(col).strip() for col in df.columns.values]

# df['Mid_Price'] = (df['Price Ask'] + df['Price Bid']) / 2
# df['Mid_Price_Future'] = df['Mid_Price'].shift(-1)
df = df.reset_index()

# Convert 'Timestamp' to a timedelta and add it to 'Date' to get a datetime
df['Datetime'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df['Timestamp'], unit='s')

# Set this new datetime as the index
df.set_index('Datetime', inplace=True)

# drop Exchange, Timestamp and Date columns
df.drop(['Exchange', 'Timestamp', 'Date'], axis=1, inplace=True)

window_size = '1min'

df = df.resample(window_size).agg({
    'Price Ask': 'mean',
    'Price Bid': 'mean',
    'Quantity Ask': 'mean',
    'Quantity Bid': 'mean'
    })

df.dropna()

df['mean_market_spread'] = df['Price Ask'] - df['Price Bid']

df['mean_quantity_spread'] = df['Quantity Ask'] - df['Quantity Bid']


df

,Price Ask,Price Bid,Quantity Ask,Quantity Bid,mean_market_spread,mean_quantity_spread
Datetime,,,,,,
2025-01-02 00:00:00,384.482537,205.366348,3.504153,4.563466,179.116188,-1.059313
2025-01-02 00:01:00,399.636529,198.025495,2.696622,4.721517,201.611035,-2.024896
2025-01-02 00:02:00,408.723619,212.040374,2.687264,5.204874,196.683244,-2.517610
2025-01-02 00:03:00,382.039178,211.957824,3.248234,4.969774,170.081354,-1.721539
2025-01-02 00:04:00,401.283418,199.870303,3.080907,4.415017,201.413115,-1.334110
...,...,...,...,...,...,...
2025-01-02 08:08:00,423.796779,216.919463,2.970598,3.785372,206.877316,-0.814773
2025-01-02 08:09:00,444.680696,227.595659,3.365487,4.971907,217.085037,-1.606420
2025-01-02 08:10:00,444.115153,224.418824,3.232828,4.173725,219.696329,-0.940897
